In [1]:
import re
import time

import numpy as np
import pandas as pd
import requests

In [2]:
# !pip3 install progressbar2
from progressbar import progressbar

## Scraping

### Scraping Functions

In [3]:
# function to scrape reddit page (takes a reddit .json url)
# returns posts 

def scraper_bike(url):
    headers = {'User-Agent' : 'override this bad boy!'}
    posts = []
    after = {}

    for page in progressbar(range(40)):
        params = {'after': after}
        pagepull = requests.get(url=url, params=params, headers=headers)
        page_dict = pagepull.json()
        posts.extend(page_dict['data']['children'])
        after = page_dict['data']['after']
        time.sleep(.2)
        
    return posts

In [4]:
# function to convert posts to DataFrame - won't allow duplicate posts since unique id 'name' is set as index
# Extract: name (as index) and subreddit, selftext, title (as columns)

def posts_to_df(post_list):
    i = 0
    post_dict = {}
    
    for post in post_list:
        ind = post_list[i]['data']
        post_dict[ind['name']] = [ind['subreddit'], ind['title'], ind['selftext']]
        i += 1

    df_name = pd.DataFrame(post_dict)
    df_name = df_name.T
    df_name.columns = ['subreddit', 'title', 'selftext']
    
    return df_name

In [5]:
# takes scraper function and url - outputs dataframe

def scrape_to_df(scrape_func, url):
    
    return posts_to_df(scrape_func(url))

In [6]:
#### If you want to scrape repeatedly over time and add to a csv
# scrape, import csv, concat, drop duplicate, and output to csv
# takes in scraper function, url, csv filename to import, csv filename to output
# Outputs - Concatenated DataFrame as csv

def scrape_add(scrape_func, url, import_file, export_file):
    scrape_df = posts_to_df(scrape_func(url))
    imported_df = pd.read_csv(import_file, index_col = 'Unnamed: 0')
    concat_df = pd.concat([imported_df, scrape_df])
    concat_df = concat_df[~concat_df.index.duplicated(keep='first')]
    concat_df.to_csv(export_file)

### Run Scrape

In [7]:
# You can also put in any 2 subreddits in as the URL and get results for those

nfltest = scrape_to_df(scraper_bike, 'https://www.reddit.com/r/nfl.json')
nbatest = scrape_to_df(scraper_bike, 'https://www.reddit.com/r/nba.json')

100% (40 of 40) |########################| Elapsed Time: 0:00:25 Time:  0:00:25
100% (40 of 40) |########################| Elapsed Time: 0:00:28 Time:  0:00:28


In [8]:
politics_test = scrape_to_df(scraper_bike, 'https://www.reddit.com/r/politics.json')
conservative_test = scrape_to_df(scraper_bike, 'https://www.reddit.com/r/conservative.json')

100% (40 of 40) |########################| Elapsed Time: 0:00:26 Time:  0:00:26
100% (40 of 40) |########################| Elapsed Time: 0:00:24 Time:  0:00:24


In [9]:
nbatest.shape

(785, 3)

In [10]:
nfltest.shape

(912, 3)

##### These scrape_add functions add to already built csvs

In [14]:
# scrape_add(scraper_bike, 'https://www.reddit.com/r/CollegeBasketball/new.json', 'NCAA_Posts_Update2.csv', 'NCAA_Posts_Update3.csv')
# scrape_add(scraper_bike, 'https://www.reddit.com/r/AskScience/new.json', 'AskSci_Posts_Update2.csv', 'AskSci_Posts_Update3.csv')
# scrape_add(scraper_bike, 'https://www.reddit.com/r/nba/new.json', 'NBA_Posts_Update2.csv', 'NBA_Posts_Update3.csv')
# scrape_add(scraper_bike, 'https://www.reddit.com/r/nfl/new.json', 'NFL_Posts_Update2.csv', 'NFL_Posts_Update3.csv')

### Data Cleaning / Preprocessing

In [15]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer, sent_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

pd.set_option('max_colwidth', 300)

In [16]:
# drop column

nfltest = nfltest.drop(columns = 'selftext')
nbatest = nbatest.drop(columns = 'selftext')

In [17]:
# merge subreddit data

train = pd.concat([nfltest, nbatest])

In [18]:
train

,subreddit,title
t3_f2b7iv,nfl,Talko Tuesday
t3_f1ucwm,nfl,r/NFL Top 100 of the 2019 Season - CALL FOR RANKERS
t3_f2sslg,nfl,[Withers] AP source says Browns star DE Myles Garrett to be reinstated by NFL from suspension today.
t3_f2i0bo,nfl,[Emmanuel Sanders] 😂 Its crazy how people keep talking bout my age but I’m still flying past 23 year olds and only had 2 drops the entire year. 1 with the broncos and one with the niners. Keep bringing up my age to make yourself feel good but go look at the film.
t3_f2r86r,nfl,"Vikings OC Gary Kubiak excited to be back calling plays: “I never lost the joy. I just said, 'Coach, do you mind if I go back home and mow some grass for a couple days or something to think about it?’ I think I called him in about a day and a half. I was just really excited to do it."""
...,...,...
t3_f1s91b,nba,What seed/how far would this team go in the West if they were coached by Brad Stevens
t3_f1m0cb,nba,People need to understand the NBA and team sports in general are about matchups.
t3_f1nv9u,nba,Do you think the Raptors have a shot of taking out the Bucks in the playoffs?
t3_f20oyn,nba,"Other than Lakers, Clippers, and Bucks, who else has a solid chance at the finals?"


##### Tokenize (grab only word characters)

In [19]:
word_tokenizer = RegexpTokenizer(r'\w+')

In [20]:
train['title'] = train['title'].map(lambda x: word_tokenizer.tokenize(x.lower()))

In [21]:
# rejoin list of tokenized words into single string for each row

train['title'] = train['title'].map(lambda x: ' '.join(x))

In [24]:
train.head()

,subreddit,title
t3_f2b7iv,nfl,talko tuesday
t3_f1ucwm,nfl,r nfl top 100 of the 2019 season call for rankers
t3_f2sslg,nfl,withers ap source says browns star de myles garrett to be reinstated by nfl from suspension today
t3_f2i0bo,nfl,emmanuel sanders its crazy how people keep talking bout my age but i m still flying past 23 year olds and only had 2 drops the entire year 1 with the broncos and one with the niners keep bringing up my age to make yourself feel good but go look at the film
t3_f2r86r,nfl,vikings oc gary kubiak excited to be back calling plays i never lost the joy i just said coach do you mind if i go back home and mow some grass for a couple days or something to think about it i think i called him in about a day and a half i was just really excited to do it


In [23]:
train['title'][0:5]

t3_f2b7iv                                                                                                                                                                                                                                                                         talko tuesday
t3_f1ucwm                                                                                                                                                                                                                                     r nfl top 100 of the 2019 season call for rankers
t3_f2sslg                                                                                                                                                                                     withers ap source says browns star de myles garrett to be reinstated by nfl from suspension today
t3_f2i0bo                      emmanuel sanders its crazy how people keep talking bout my age but i m still flying past 23 year olds and

### Train test split and converting series to list of strings then to array

In [25]:
X = train[['title']]
y = train['subreddit']

In [26]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=.25,
                                                    random_state=42,
                                                    stratify=y)

In [27]:
# baseline is

y.value_counts(normalize=True)

nfl    0.537419
nba    0.462581
Name: subreddit, dtype: float64

In [28]:
# create our training data list - this is a list of strings, with each string being a post title

clean_train_data = []

for traindata in X_train['title']:
    clean_train_data.append(traindata)

In [32]:
len(clean_train_data)

1272

In [33]:
clean_train_data[:10]

['raiders on this day 51 years ago al davis promoted the then linebackers coach john madden to head coach the rest is history',
 'nick young or j r',
 'free talk friday',
 'why don t nfl players offload lateral pass more',
 'the business of being andre iguodala nba countdown',
 'kd or kyrie who s taking the last second shot to win the game',
 'jim irsay colts owner possibly hinting at andrew luck return',
 'rapoport free agent te greg olsen has a redskins visit through today source said he arrived yesterday his seahawks visit is wednesday and then olsen expects a decision by the end of the week olsen who was in the booth for an xfl game yesterday has also spent time with the bills',
 'how many nfl teams could peyton manning start for at this moment',
 'ray allen or manu ginobili']

In [30]:
# create test data list

clean_test_data = []

for testdata in X_test['title']:
    clean_test_data.append(testdata)

In [31]:
len(clean_test_data)

425

### Count Vectorizer

In [34]:
# instantiate our CountVectorizer. This counts the number of appearances of all the words in our training data and
# eliminates common english stop words. 5000 max features works well for our purposes (tested various numbers). Our
# data is already preprocessed and tokenized manually earlier. ngram_range is 1,3, although all or nearly all our
# features are single words

vectorizer = CountVectorizer(analyzer="word",
                             tokenizer=None,
                             preprocessor=None,
                             stop_words='english',
                             max_features=5000,
                             ngram_range=(1, 3))

In [35]:
# fit our training data and test data lists to our count_vectorizer

train_data_features = vectorizer.fit_transform(clean_train_data)

test_data_features = vectorizer.transform(clean_test_data)

In [36]:
# convert to array

train_data_features = train_data_features.toarray()

In [37]:
# check shapes

train_data_features.shape, test_data_features.shape

((1272, 5000), (425, 5000))

In [38]:
# I wanted check that the features corpus was as expected - removed print statement for readability

vocab = vectorizer.get_feature_names()

In [39]:
vocab[0:200]

['000',
 '02',
 '02 06',
 '06',
 '07',
 '10',
 '10 2020',
 '10 fgm',
 '10 game',
 '10 game span',
 '10 games',
 '10 nfl',
 '10 points',
 '10 seasons',
 '10 year',
 '10 years',
 '100',
 '100 nfl',
 '101',
 '101 players',
 '101 players 2019',
 '103',
 '105',
 '106',
 '10th',
 '10th straight',
 '11',
 '11 2020',
 '11 points',
 '111',
 '113',
 '114',
 '116',
 '116 105',
 '11th',
 '12',
 '120',
 '123',
 '126',
 '13',
 '133',
 '133 92',
 '135',
 '14',
 '140',
 '142',
 '15',
 '15 39',
 '15 game',
 '15 game winning',
 '15 games',
 '15th',
 '16',
 '17',
 '17 18',
 '18',
 '18 33',
 '18 million',
 '18 points',
 '19',
 '19 35',
 '1966',
 '1980',
 '1980 1984 different',
 '1980 81',
 '1980 81 21',
 '1981',
 '1981 49ers',
 '1981 49ers falcons',
 '1981 nfl',
 '1981 nfl today',
 '1983',
 '1983 houston',
 '1983 houston rockets',
 '1984',
 '1984 different',
 '1984 different franchises',
 '1986',
 '1986 game',
 '1986 game giants',
 '1986 nfl',
 '1986 nfl mvp',
 '1988',
 '1988 dunk',
 '1988 dunk contest',


## MODELING

### Logistic Regression

In [40]:
from sklearn.linear_model import LogisticRegression

In [51]:
# fit logistic regression model

lr = LogisticRegression(l1_ratio=True)

In [52]:
# shape check

train_data_features.shape, y_train.shape

((1272, 5000), (1272,))

In [53]:
lr.fit(train_data_features, y_train)

/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1506: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  "(penalty={})".format(self.penalty))


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=True, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [54]:
lr.score(train_data_features, y_train)

0.9960691823899371

In [55]:
lr.score(test_data_features, y_test)

0.88

### Feature comparison

Creates a dataframe that matches features to coefficients

In [56]:
coef_list = lr.coef_.tolist()

In [57]:
coef_list = coef_list[0]

In [58]:
coef_df = pd.DataFrame({'features': vectorizer.get_feature_names(),
                        'coefs': coef_list})

In [59]:
coef_df.sort_values(by = ['coefs'])

,features,coefs
1980,nba,-2.182772
1141,embiid,-1.271563
1700,knicks,-1.236901
4997,zion,-1.141094
4901,westbrook,-1.107876
...,...,...
828,chiefs,1.220847
4578,super,1.309942
1842,mahomes,1.416220
3112,qb,1.459775


### Let's throw out these unfair words and rerun

In [60]:
stopwords = set(stopwords.words('english'))

extra_stopwords = ['nba', 'basketball', 'football', 'nfl']

stopwords.update(extra_stopwords)

In [61]:
vectorizer = CountVectorizer(analyzer = "word",
                             tokenizer = None,
                             preprocessor = None,
                             stop_words = stopwords,
                             max_features = 5000,
                             ngram_range = (1, 3))

train_data_features = vectorizer.fit_transform(clean_train_data)

test_data_features = vectorizer.transform(clean_test_data)

train_data_features = train_data_features.toarray()

train_data_features.shape, test_data_features.shape

((1272, 5000), (425, 5000))

In [62]:
lr.fit(train_data_features, y_train)

/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1506: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  "(penalty={})".format(self.penalty))


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=True, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [63]:
lr.score(train_data_features, y_train)

0.9905660377358491

In [64]:
lr.score(test_data_features, y_test)

0.8658823529411764

In [71]:
coef_list = lr.coef_.tolist()
coef_list = coef_list[0]

coef_df = pd.DataFrame({'features' : vectorizer.get_feature_names(),
                       'coefs' : coef_list})

coef_df.sort_values(by = ['coefs'])[2300:2400]

,features,coefs
3122,qb dak prescott,0.003643
3465,rapoport cowboys going,0.003643
2703,prescott close,0.003643
846,chiefs fan,0.003654
2697,preparing speech share,0.003803
...,...,...
2226,pelissero chiefs,0.005248
3451,rap 49ers,0.005460
3452,rap 49ers try,0.005460
201,2020 enter,0.005468


### Decision Tree

In [72]:
from sklearn.tree import DecisionTreeClassifier

In [73]:
tree = DecisionTreeClassifier()

In [74]:
tree.fit(train_data_features, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [75]:
tree.score(train_data_features, y_train)

1.0

In [76]:
tree.score(test_data_features, y_test)

0.7294117647058823

### Random Forest

In [77]:
from sklearn.ensemble import RandomForestClassifier

In [78]:
forest = RandomForestClassifier(n_estimators = 100)

In [79]:
forest.fit(train_data_features, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [80]:
forest.score(train_data_features, y_train)

1.0

In [81]:
forest.score(test_data_features, y_test)

0.851764705882353

###  Matrix on Logistic Regression

In [82]:
from sklearn.metrics import confusion_matrix

In [83]:
y_pred = lr.predict(test_data_features)

In [84]:
cm = confusion_matrix(y_test, y_pred)

In [85]:
cm_df = pd.DataFrame(cm,
                    columns=['predict_neg', 'predict_pos'],
                    index = ['actual_neg', 'actual_pos'])

In [86]:
cm_df

,predict_neg,predict_pos
actual_neg,178,19
actual_pos,38,190


## Checking where our model failed

In [87]:
comparison_df = pd.DataFrame({'y_actual' : y_test,
             'y_predicted' : y_pred})

In [88]:
mismatch_df = comparison_df[comparison_df['y_actual'] != comparison_df['y_predicted']]

In [89]:
mismatch2_df = pd.concat([mismatch_df, X_test], axis = 1)

/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [90]:
# All incorrect predictions with titles

mismatches = mismatch2_df.dropna()

In [91]:
mismatches

,y_actual,y_predicted,title
t3_exx143,nfl,nba,pelissero both teams lined up at the 24 yard line a tribute to kobe bryant
t3_exy1ns,nfl,nba,highlight chiefs trick play on 4th and 1
t3_ey09cu,nfl,nba,highlight williams runs it in for the td
t3_ey0bjo,nfl,nba,highlight jimmy g overthrows emmanuel sanders deep under 2 minutes
t3_eylebl,nfl,nba,highlight josh rosen airs out a perfect spiral to christian kirk on the first play of the game a 75 yard td pass
t3_eyt34u,nfl,nba,talko tuesday
t3_eyvlk8,nfl,nba,think back to your pre season predictions what did you get wrong
t3_eywhh4,nfl,nba,nfl player kurt warner getting movie
t3_ez105p,nfl,nba,pfn there are many in the scouting community that think chase young is overrated
t3_ez2hw6,nfl,nba,which right on the bridge of hof players would make canton if they got one lombardi trophy


### Let's try TF-IDF

Term Frequency / Inverse Document Frequency

TF(w) = (Number of times term w appears in a document) / (Total number of terms in the document)

IDF(w) = log_e(Total number of documents / Number of documents with term w in it)

In [92]:
tfidf_vec = TfidfVectorizer(analyzer="word",
                            tokenizer=None,
                            preprocessor=None,
                            stop_words=['nba', 'nfl', 'football', 'basketball'],
                            max_features=5000,
                            ngram_range=(1, 3))

In [93]:
train_data_features = tfidf_vec.fit_transform(clean_train_data)

test_data_features = tfidf_vec.transform(clean_test_data)

train_data_features = train_data_features.toarray()

train_data_features.shape, test_data_features.shape

((1272, 5000), (425, 5000))

In [94]:
lr.fit(train_data_features, y_train)

/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1506: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  "(penalty={})".format(self.penalty))


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=True, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [95]:
lr.score(train_data_features, y_train)

0.97562893081761

In [96]:
lr.score(test_data_features, y_test)

0.8470588235294118

### Let's try on some other subreddits

In [97]:
train = pd.concat([politics_test, conservative_test])

In [98]:
X = train[['title']]
y = train['subreddit']

In [99]:
# politics_test = scrape_to_df(scraper_bike, 'https://www.reddit.com/r/politics.json')
# conservative_test = scrape_to_df(scraper_bike, 'https://www.reddit.com/r/conservative.json')

In [100]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, stratify = y)

In [101]:
politics_test = politics_test.drop(columns='selftext')
conservative_test = conservative_test.drop(columns='selftext')

train = pd.concat([politics_test, conservative_test])
tokenizer = RegexpTokenizer(r'\w+')

train['title'] = train['title'].map(lambda x: tokenizer.tokenize(x.lower()))
train['title'] = train['title'].map(lambda x: ' '.join(x))

In [102]:
# create our training data list - this is a list of strings, with each string being a post title

clean_train_data = []

for traindata in X_train['title']:
    clean_train_data.append(traindata)
    
    
# create test data list

clean_test_data = []

for testdata in X_test['title']:
    clean_test_data.append(testdata)

In [103]:
vectorizer = CountVectorizer(analyzer="word",
                             tokenizer=None,
                             preprocessor=None,
                             stop_words='english',
                             max_features=5000,
                             ngram_range=(1, 3))

train_data_features = vectorizer.fit_transform(clean_train_data)

test_data_features = vectorizer.transform(clean_test_data)

train_data_features = train_data_features.toarray()

train_data_features.shape, test_data_features.shape

vocab = vectorizer.get_feature_names()

### Modeling

In [104]:
lr = LogisticRegression(penalty = 'l2')

In [105]:
train_data_features.shape, y_train.shape

((1297, 5000), (1297,))

In [106]:
lr.fit(train_data_features, y_train)

lr.score(train_data_features, y_train)

/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.9838087895142636

In [107]:
lr.score(test_data_features, y_test)

0.7390300230946882

In [108]:
coef_list = lr.coef_.tolist()

coef_list = coef_list[0]

In [109]:
coef_df = pd.DataFrame({'features' : vectorizer.get_feature_names(),
                       'coefs' : coef_list})

coef_df.sort_values(by = ['coefs'])

,features,coefs
702,dems,-1.385459
3943,socialism,-1.172609
3974,spending,-1.104884
1911,people,-1.032575
4324,video,-1.001417
...,...,...
318,bloomberg,1.033142
1521,military,1.042041
4005,stone,1.249537
4183,trump,1.259295


# Understanding word vectors

... for, like, actual poets. By [Allison Parrish](http://www.decontextualize.com/)


## Why word vectors?

Poetry is, at its core, the art of identifying and manipulating linguistic similarity. I have discovered a truly marvelous proof of this, which this notebook is too narrow to contain. (By which I mean: I will elaborate on this some other time)

## Animal similarity and simple linear algebra

We'll begin by considering a small subset of English: words for animals. Our task is to be able to write computer programs to find similarities among these words and the creatures they designate. To do this, we might start by making a spreadsheet of some animals and their characteristics. For example:

![Animal spreadsheet](http://static.decontextualize.com/snaps/animal-spreadsheet.png)

This spreadsheet associates a handful of animals with two numbers: their cuteness and their size, both in a range from zero to one hundred. (The values themselves are simply based on my own judgment. Your taste in cuteness and evaluation of size may differ significantly from mine. As with all data, these data are simply a mirror reflection of the person who collected them.)

These values give us everything we need to make determinations about which animals are similar (at least, similar in the properties that we've included in the data). Try to answer the following question: Which animal is most similar to a capybara? You could go through the values one by one and do the math to make that evaluation, but visualizing the data as points in 2-dimensional space makes finding the answer very intuitive:

![Animal space](http://static.decontextualize.com/snaps/animal-space.png)

The plot shows us that the closest animal to the capybara is the panda bear (again, in terms of its subjective size and cuteness). One way of calculating how "far apart" two points are is to find their *Euclidean distance*. (This is simply the length of the line that connects the two points.) For points in two dimensions, Euclidean distance can be calculated with the following Python function:

In [110]:
import numpy as np
def distance2d(x1, y1, x2, y2):
    return np.linalg.norm(np.array([x1, y1])-np.array([x2, y2]))

So, the distance between "capybara" (70, 30) and "panda" (74, 40):

In [111]:
distance2d(70, 30, 75, 40) # panda and capybara

11.180339887498949

... is less than the distance between "tarantula" and "elephant":

In [112]:
distance2d(8, 3, 65, 90) # tarantula and elephant

104.0096149401583

Modeling animals in this way has a few other interesting properties. For example, you can pick an arbitrary point in "animal space" and then find the animal closest to that point. If you imagine an animal of size 25 and cuteness 30, you can easily look at the space to find the animal that most closely fits that description: the chicken.

Reasoning visually, you can also answer questions like: what's halfway between a chicken and an elephant? Simply draw a line from "elephant" to "chicken," mark off the midpoint and find the closest animal. (According to our chart, halfway between an elephant and a chicken is a horse.)

You can also ask: what's the *difference* between a hamster and a tarantula? According to our plot, it's about seventy five units of cute (and a few units of size).

The relationship of "difference" is an interesting one, because it allows us to reason about *analogous* relationships. In the chart below, I've drawn an arrow from "tarantula" to "hamster" (in blue):

![Animal analogy](http://static.decontextualize.com/snaps/animal-space-analogy.png)

You can understand this arrow as being the *relationship* between a tarantula and a hamster, in terms of their size and cuteness (i.e., hamsters and tarantulas are about the same size, but hamsters are much cuter). In the same diagram, I've also transposed this same arrow (this time in red) so that its origin point is "chicken." The arrow ends closest to "kitten." What we've discovered is that the animal that is about the same size as a chicken but much cuter is... a kitten. To put it in terms of an analogy:

    Tarantulas are to hamsters as chickens are to kittens.
    
A sequence of numbers used to identify a point is called a *vector*, and the kind of math we've been doing so far is called *linear algebra.* (Linear algebra is surprisingly useful across many domains: It's the same kind of math you might do to, e.g., simulate the velocity and acceleration of a sprite in a video game.)

A set of vectors that are all part of the same data set is often called a *vector space*. The vector space of animals in this section has two *dimensions*, by which I mean that each vector in the space has two numbers associated with it (i.e., two columns in the spreadsheet). The fact that this space has two dimensions just happens to make it easy to *visualize* the space by drawing a 2D plot. But most vector spaces you'll work with will have more than two dimensions—sometimes many hundreds. In those cases, it's more difficult to visualize the "space," but the math works pretty much the same.

## Language with vectors: colors

So far, so good. We have a system in place—albeit highly subjective—for talking about animals and the words used to name them. I want to talk about another vector space that has to do with language: the vector space of colors.

Colors are often represented in computers as vectors with three dimensions: red, green, and blue. Just as with the animals in the previous section, we can use these vectors to answer questions like: which colors are similar? What's the most likely color name for an arbitrarily chosen set of values for red, green and blue? Given the names of two colors, what's the name of those colors' "average"?

We'll be working with this [color data](https://github.com/dariusk/corpora/blob/master/data/colors/xkcd.json) from the [xkcd color survey](https://blog.xkcd.com/2010/05/03/color-survey-results/). The data relates a color name to the RGB value associated with that color. [Here's a page that shows what the colors look like](https://xkcd.com/color/rgb/). Download the color data and put it in the same directory as this notebook.

A few notes before we proceed:

* The linear algebra functions implemented below (`addv`, `meanv`, etc.) are slow, potentially inaccurate, and shouldn't be used for "real" code—I wrote them so beginner programmers can understand how these kinds of functions work behind the scenes. Use [numpy](http://www.numpy.org/) for fast and accurate math in Python.
* If you're interested in perceptually accurate color math in Python, consider using the [colormath library](http://python-colormath.readthedocs.io/en/latest/).

Now, import the `json` library and load the color data:

In [113]:
import requests

In [114]:
resp = requests.get('https://raw.githubusercontent.com/dariusk/corpora/master/data/colors/xkcd.json')
color_data = resp.json()

The following function converts colors from hex format (`#1a2b3c`) to a tuple of integers:

In [121]:
def hex_to_int(s):
    return int(s[1:3], 16), int(s[3:5], 16), int(s[5:7], 16)

And the following cell creates a dictionary and populates it with mappings from color names to RGB vectors for each color in the data:

In [122]:
colors = dict()
for item in color_data['colors']:
    colors[item["color"]] = hex_to_int(item["hex"])

Testing it out:

In [123]:
colors['olive']

(110, 117, 14)

In [124]:
colors['red']

(229, 0, 0)

In [119]:
colors['black']

(0, 0, 0)

In [120]:
colors.keys()

dict_keys(['cloudy blue', 'dark pastel green', 'dust', 'electric lime', 'fresh green', 'light eggplant', 'nasty green', 'really light blue', 'tea', 'warm purple', 'yellowish tan', 'cement', 'dark grass green', 'dusty teal', 'grey teal', 'macaroni and cheese', 'pinkish tan', 'spruce', 'strong blue', 'toxic green', 'windows blue', 'blue blue', 'blue with a hint of purple', 'booger', 'bright sea green', 'dark green blue', 'deep turquoise', 'green teal', 'strong pink', 'bland', 'deep aqua', 'lavender pink', 'light moss green', 'light seafoam green', 'olive yellow', 'pig pink', 'deep lilac', 'desert', 'dusty lavender', 'purpley grey', 'purply', 'candy pink', 'light pastel green', 'boring green', 'kiwi green', 'light grey green', 'orange pink', 'tea green', 'very light brown', 'egg shell', 'eggplant purple', 'powder pink', 'reddish grey', 'baby shit brown', 'liliac', 'stormy blue', 'ugly brown', 'custard', 'darkish pink', 'deep brown', 'greenish beige', 'manilla', 'off blue', 'battleship gre

### Vector math

Before we keep going, we'll need some functions for performing basic vector arithmetic. These functions will work with vectors in spaces of any number of dimensions.

The first function returns the Euclidean distance between two points:

In [125]:
import math
def distance(coord1, coord2):
    return np.linalg.norm(np.array(coord1)-np.array(coord2))
distance([10, 1], [5, 2])

5.0990195135927845

The `subtractv` function subtracts one vector from another:

In [126]:
def subtractv(coord1, coord2):
    return list(np.array(coord1) - np.array(coord2))
subtractv([10, 1], [5, 2])

[5, -1]

The `addv` vector adds two vectors together:

In [127]:
def addv(coord1, coord2):
    return list(np.array(coord1) + np.array(coord2))
addv([10, 1], [5, 2])

[15, 3]

And the `meanv` function takes a list of vectors and finds their mean or average:

In [128]:
def meanv(coords):
    return list(np.mean(coords, axis=0))
meanv([[0, 1], [2, 2], [4, 3]])

[2.0, 2.0]

Just as a test, the following cell shows that the distance from "red" to "green" is greater than the distance from "red" to "pink":

In [129]:
distance(colors['red'], colors['green']) > distance(colors['red'], colors['pink'])

True

### Finding the closest item

Just as we wanted to find the animal that most closely matched an arbitrary point in cuteness/size space, we'll want to find the closest color name to an arbitrary point in RGB space. The easiest way to find the closest item to an arbitrary vector is simply to find the distance between the target vector and each item in the space, in turn, then sort the list from closest to farthest. The `closest()` function below does just that. By default, it returns a list of the ten closest items to the given vector.

> Note: Calculating "closest neighbors" like this is fine for the examples in this notebook, but unmanageably slow for vector spaces of any appreciable size. As your vector space grows, you'll want to move to a faster solution, like SciPy's [kdtree](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.spatial.KDTree.html) or [Annoy](https://pypi.python.org/pypi/annoy).

In [130]:
def closest(space, coord, n=10):
    return sorted(space.keys(), key=lambda x: distance(coord, space[x]))[:n]

Testing it out, we can find the ten colors closest to "red":

In [131]:
closest(colors, colors['white'])

['white',
 'pale grey',
 'very light pink',
 'off white',
 'ice blue',
 'very pale blue',
 'ice',
 'very light blue',
 'really light blue',
 'eggshell']

... or the ten colors closest to (150, 60, 150):

In [132]:
closest(colors, [255, 60, 150])

['barbie pink',
 'warm pink',
 'medium pink',
 'rosy pink',
 'darkish pink',
 'bubble gum pink',
 'strong pink',
 'electric pink',
 'bubblegum',
 'hot pink']

### Color magic

The magical part of representing words as vectors is that the vector operations we defined earlier appear to operate on language the same way they operate on numbers. For example, if we find the word closest to the vector resulting from subtracting "red" from "purple," we get a series of "blue" colors:

In [133]:
closest(colors, subtractv(colors['green'], colors['purple']))

['true green',
 'racing green',
 'bottle green',
 'deep green',
 'darkgreen',
 'forest',
 'emerald green',
 'dark green',
 'vibrant green',
 'british racing green']

This matches our intuition about RGB colors, which is that purple is a combination of red and blue. Take away the red, and blue is all you have left.

You can do something similar with addition. What's blue plus green?

In [ ]:
closest(colors, addv(colors['blue'], colors['green']))

That's right, it's something like turquoise or cyan! What if we find the average of black and white? Predictably, we get gray:

In [ ]:
# the average of black and white: medium grey
closest(colors, meanv([colors['black'], colors['white']]))

Just as with the tarantula/hamster example from the previous section, we can use color vectors to reason about relationships between colors. In the cell below, finding the difference between "pink" and "red" then adding it to "blue" seems to give us a list of colors that are to blue what pink is to red (i.e., a slightly lighter, less saturated shade):

In [ ]:
# an analogy: pink is to red as X is to blue
pink_to_red = subtractv(colors['orange'], colors['red'])
closest(colors, addv(pink_to_red, colors['blue']))

Another example of color analogies: Navy is to blue as true green/dark grass green is to green:

In [ ]:
# another example: 
navy_to_blue = subtractv(colors['navy'], colors['blue'])
closest(colors, addv(navy_to_blue, colors['green']))

The examples above are fairly simple from a mathematical perspective but nevertheless *feel* magical: they're demonstrating that it's possible to use math to reason about how people use language.

### Interlude: A Love Poem That Loses Its Way

In [ ]:
import random
red = colors['red']
blue = colors['blue']
for i in range(14):
    rednames = closest(colors, red)
    bluenames = closest(colors, blue)
    print(f"Roses are {rednames[0]}, violets are {bluenames[0]}")
    red = colors[random.choice(rednames[1:])]
    blue = colors[random.choice(bluenames[1:])]

### Doing bad digital humanities with color vectors

With the tools above in hand, we can start using our vectorized knowledge of language toward academic ends. In the following example, I'm going to calculate the average color of Bram Stoker's *Dracula*.

First, we'll load [spaCy](https://spacy.io/):

In [136]:
import spacy
!python -m spacy download en_core_web_sm
nlp = spacy.load('en_core_web_sm')

     |████████████████████████████████| 12.0MB 3.6MB/s eta 0:00:01
  Created wheel for en-core-web-sm: filename=en_core_web_sm-2.2.5-cp36-none-any.whl size=12011741 sha256=b59fb35c43748c1641a94fa92b67bbf099f41d60f104d91eb1ce51845e874dd0
  Stored in directory: /private/var/folders/_g/tq7t9jvs7s1dj4r_7p4ybbwr0000gn/T/pip-ephem-wheel-cache-gxq_k93f/wheels/6a/47/fb/6b5a0b8906d8e8779246c67d4658fd8a544d4a03a75520197a
Successfully built en-core-web-sm
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.

To calculate the average color, we'll follow these steps:

1. Parse the text into words
2. Check every word to see if it names a color in our vector space. If it does, add it to a list of vectors.
3. Find the average of that list of vectors.
4. Find the color(s) closest to that average vector.

The following cell performs steps 1-3:

In [ ]:
resp = requests.get('http://www.gutenberg.org/cache/epub/345/pg345.txt')
dracula = nlp(resp.text)
# use word.lower_ to normalize case
drac_colors = [colors[word.lower_] for word in dracula if word.lower_ in colors]
avg_color = meanv(drac_colors)
print(avg_color)

Now, we'll pass the averaged color vector to the `closest()` function, yielding... well, it's just a brown mush, which is kinda what you'd expect from adding a bunch of colors together willy-nilly.

In [ ]:
closest(colors, avg_color)

On the other hand, here's what we get when we average the colors of Charlotte Perkins Gilman's classic *The Yellow Wallpaper*. The result definitely reflects the content of the story, so maybe we're on to something here.

In [ ]:
resp = requests.get('http://www.gutenberg.org/cache/epub/1952/pg1952.txt')
yellow = nlp(resp.text)
wallpaper_colors = [colors[word.lower_] for word in yellow if word.lower_ in colors]
avg_color = meanv(wallpaper_colors)
closest(colors, avg_color)

Exercise for the reader: Use the vector arithmetic functions to rewrite a text, making it...

* more blue (i.e., add `colors['blue']` to each occurrence of a color word); or
* more light (i.e., add `colors['white']` to each occurrence of a color word); or
* darker (i.e., attenuate each color. You might need to write a vector multiplication function to do this one right.)

## Distributional semantics

In the previous section, the examples are interesting because of a simple fact: colors that we think of as similar are "closer" to each other in RGB vector space. In our color vector space, or in our animal cuteness/size space, you can think of the words identified by vectors close to each other as being *synonyms*, in a sense: they sort of "mean" the same thing. They're also, for many purposes, *functionally identical*. Think of this in terms of writing, say, a search engine. If someone searches for "mauve trousers," then it's probably also okay to show them results for, say,

In [ ]:
for cname in closest(colors, colors['mauve']):
    print(cname, "trousers")

That's all well and good for color words, which intuitively seem to exist in a multidimensional continuum of perception, and for our animal space, where we've written out the vectors ahead of time. But what about... arbitrary words? Is it possible to create a vector space for all English words that has this same "closer in space is closer in meaning" property?

To answer that, we have to back up a bit and ask the question: what does *meaning* mean? No one really knows, but one theory popular among computational linguists, computer scientists and other people who make search engines is the [Distributional Hypothesis](https://en.wikipedia.org/wiki/Distributional_semantics), which states that:

    Linguistic items with similar distributions have similar meanings.
    
What's meant by "similar distributions" is *similar contexts*. Take for example the following sentences:

    It was really cold yesterday.
    It will be really warm today, though.
    It'll be really hot tomorrow!
    Will it be really cool Tuesday?
    
According to the Distributional Hypothesis, the words `cold`, `warm`, `hot` and `cool` must be related in some way (i.e., be close in meaning) because they occur in a similar context, i.e., between the word "really" and a word indicating a particular day. (Likewise, the words `yesterday`, `today`, `tomorrow` and `Tuesday` must be related, since they occur in the context of a word indicating a temperature.)

In other words, according to the Distributional Hypothesis, a word's meaning is just a big list of all the contexts it occurs in. Two words are closer in meaning if they share contexts.

## Word vectors by counting contexts

So how do we turn this insight from the Distributional Hypothesis into a system for creating general-purpose vectors that capture the meaning of words? Maybe you can see where I'm going with this. What if we made a *really big* spreadsheet that had one column for every context for every word in a given source text. Let's use a small source text to begin with, such as this excerpt from Dickens:

    It was the best of times, it was the worst of times.

Such a spreadsheet might look something like this:

![dickens contexts](http://static.decontextualize.com/snaps/best-of-times.png)

The spreadsheet has one column for every possible context, and one row for every word. The values in each cell correspond with how many times the word occurs in the given context. The numbers in the columns constitute that word's vector, i.e., the vector for the word `of` is

    [0, 0, 0, 0, 1, 0, 0, 0, 1, 0]
    
Because there are ten possible contexts, this is a ten dimensional space! It might be strange to think of it, but you can do vector arithmetic on vectors with ten dimensions just as easily as you can on vectors with two or three dimensions, and you could use the same distance formula that we defined earlier to get useful information about which vectors in this space are similar to each other. In particular, the vectors for `best` and `worst` are actually the same (a distance of zero), since they occur only in the same context (`the ___ of`):

    [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
    
Of course, the conventional way of thinking about "best" and "worst" is that they're *antonyms*, not *synonyms*. But they're also clearly two words of the same kind, with related meanings (through opposition), a fact that is captured by this distributional model.

### Contexts and dimensionality

Of course, in a corpus of any reasonable size, there will be many thousands if not many millions of possible contexts. It's difficult enough working with a vector space of ten dimensions, let alone a vector space of a million dimensions! It turns out, though, that many of the dimensions end up being superfluous and can either be eliminated or combined with other dimensions without significantly affecting the predictive power of the resulting vectors. The process of getting rid of superfluous dimensions in a vector space is called [dimensionality reduction](https://en.wikipedia.org/wiki/Dimensionality_reduction), and most implementations of count-based word vectors make use of dimensionality reduction so that the resulting vector space has a reasonable number of dimensions (say, 100—300, depending on the corpus and application).

The question of how to identify a "context" is itself very difficult to answer. In the toy example above, we've said that a "context" is just the word that precedes and the word that follows. Depending on your implementation of this procedure, though, you might want a context with a bigger "window" (e.g., two words before and after), or a non-contiguous window (skip a word before and after the given word). You might exclude certain "function" words like "the" and "of" when determining a word's context, or you might [lemmatize](https://en.wikipedia.org/wiki/Lemmatisation) the words before you begin your analysis, so two occurrences with different "forms" of the same word count as the same context. These are all questions open to research and debate, and different implementations of procedures for creating count-based word vectors make different decisions on this issue.

### GloVe vectors

But you don't have to create your own word vectors from scratch! Many researchers have made downloadable databases of pre-trained vectors. One such project is Stanford's [Global Vectors for Word Representation (GloVe)](https://nlp.stanford.edu/projects/glove/). These 300-dimensional vectors are included with spaCy, and they're the vectors we'll be using for the rest of this tutorial.

## Word vectors in spaCy

Okay, let's have some fun with real word vectors. We're going to use the GloVe vectors that come with spaCy to creatively analyze and manipulate the text of Bram Stoker's *Dracula*. First, make sure you've got `spacy` imported:

In [ ]:
dracula[560]

In [ ]:
_.vector

In [137]:
# previously we've used the _sm model, which doesn't include all vectors.
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_lg-2.2.0/en_core_web_lg-2.2.0.tar.gz
import en_core_web_lg
nlp = en_core_web_lg.load()

     |█████████▍                      | 244.2MB 29kB/s eta 5:32:2094  |                                | 1.1MB 995kB/s eta 0:13:51     |▊                               | 18.7MB 2.4MB/s eta 0:05:34     |█▋                              | 41.4MB 613kB/s eta 0:21:23     |█▊                              | 44.5MB 372kB/s eta 0:35:02     |██▎                             | 57.5MB 320kB/s eta 0:40:01     |█████▏                          | 132.8MB 822kB/s eta 0:14:06     |█████▌                          | 141.0MB 443kB/s eta 0:25:50     |██████▎                         | 163.4MB 88kB/s eta 2:05:06     |██████▌                         | 168.9MB 4.5MB/s eta 0:02:26     |██████▉                         | 178.2MB 592kB/s eta 0:18:16     |████████                        | 205.6MB 638kB/s eta 0:16:15     |████████                        | 207.5MB 544kB/s eta 0:19:00     |████████▌                       | 221.2MB 430kB/s eta 0:23:29     |█████████                       | 233.0MB 1.3MB/s eta 0:07:44    

ModuleNotFoundError: No module named 'en_core_web_lg'

In [ ]:
resp = requests.get('http://www.gutenberg.org/cache/epub/345/pg345.txt')
dracula = nlp(resp.text)


And the cell below creates a list of unique words (or tokens) in the text, as a list of strings.

In [ ]:
# all of the words in the text file
tokens = list(set([w.text for w in dracula if w.is_alpha]))

You can see the vector of any word in spaCy's vocabulary using the `vocab` attribute, like so:

In [ ]:
nlp.vocab['alligator'].vector

For the sake of convenience, the following function gets the vector of a given string from spaCy's vocabulary:

In [ ]:
def vec(s):
    return nlp.vocab[s].vector

### Cosine similarity and finding closest neighbors

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

The following cell shows that the cosine similarity between `dog` and `puppy` is larger than the similarity between `trousers` and `octopus`, thereby demonstrating that the vectors are working how we expect them to:

In [ ]:
(cosine_similarity(vec('dog').reshape(1, -1), vec('puppy').reshape(1, -1))[0][0] > 
    cosine_similarity(vec('trousers').reshape(1, -1), vec('octopus').reshape(1, -1))[0][0])

The following cell defines a function that iterates through a list of tokens and returns the token whose vector is most similar to a given vector.

In [ ]:
def spacy_closest(token_list, vec_to_check, n=10):
    return sorted(token_list,
                  key=lambda x: cosine_similarity(vec_to_check.reshape(1, -1), 
                                                  vec(x).reshape(1, -1))[0][0],
                  reverse=True)[:n]

Using this function, we can get a list of synonyms, or words closest in meaning (or distribution, depending on how you look at it), to any arbitrary word in spaCy's vocabulary. In the following example, we're finding the words in *Dracula* closest to "basketball":

In [ ]:
# what's the closest equivalent of basketball?
spacy_closest(tokens, vec("basketball"))

### Fun with spaCy, Dracula, and vector arithmetic

Now we can start doing vector arithmetic and finding the closest words to the resulting vectors. For example, what word is closest to the halfway point between day and night?

In [ ]:
# halfway between day and night
spacy_closest(tokens, np.array(meanv([vec("day"), vec("night")])))

Variations of `night` and `day` are still closest, but after that we get words like `evening` and `morning`, which are indeed halfway between day and night!

Here are the closest words in _Dracula_ to "wine":

In [ ]:
spacy_closest(tokens, vec("wine"))

If you subtract "alcohol" from "wine" and find the closest words to the resulting vector, you're left with simply a lovely dinner:

In [ ]:
spacy_closest(tokens, np.array(subtractv(vec("wine"), vec("alcohol"))))

The closest words to "water":

In [ ]:
spacy_closest(tokens, vec("water"))

But if you add "frozen" to "water," you get "ice":

In [ ]:
spacy_closest(tokens, np.array(addv(vec("water"), vec("frozen"))))

You can even do analogies! For example, the words most similar to "grass":

In [ ]:
spacy_closest(tokens, vec("grass"))

If you take the difference of "blue" and "sky" and add it to grass, you get the analogous word ("green"):

In [ ]:
# analogy: blue is to sky as X is to grass
blue_to_sky = subtractv(vec("blue"), vec("sky"))
spacy_closest(tokens, np.array(addv(blue_to_sky, vec("grass"))))

## Further resources

* [Word2vec](https://en.wikipedia.org/wiki/Word2vec) is another procedure for producing word vectors which uses a predictive approach rather than a context-counting approach. [This paper](http://clic.cimec.unitn.it/marco/publications/acl2014/baroni-etal-countpredict-acl2014.pdf) compares and contrasts the two approaches. (Spoiler: it's kind of a wash.)
* If you want to train your own word vectors on a particular corpus, the popular Python library [gensim](https://radimrehurek.com/gensim/) has an implementation of Word2Vec that is relatively easy to use. [There's a good tutorial here.](https://rare-technologies.com/word2vec-tutorial/)
* When you're working with vector spaces with high dimensionality and millions of vectors, iterating through your entire space calculating cosine similarities can be a drag. I use [Annoy](https://pypi.python.org/pypi/annoy) to make these calculations faster, and you should consider using it too.